In [5]:
pip install fastf1

     ---------------------------------------- 79.4/79.4 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 48.5/48.5 kB ? eta 0:00:00
     -------------------------------------- 101.4/101.4 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [78]:
import fastf1 as ff1
from fastf1 import plotting
import pandas as pd

plotting.setup_mpl()
pd.options.mode.chained_assignment = None
#ff1.Cache.enable_cache('test/') 
year = 2021
race = 'Russia'
type = 'R'
driver = 'HAM'
race = ff1.get_session(year, race, type)
laps = race.load_laps(with_telemetry=True)

laps['LapNumber'] = laps['LapNumber'] - 1
laps = laps.loc[laps['LapNumber'] >= 45]
drivers = pd.unique(laps['Driver'])
telemetry = pd.DataFrame()

for driver in drivers:
    # print(f'driver={driver}')
    driver_laps = laps.pick_driver(driver)
    
    for lap in driver_laps.iterlaps():
        # print(f'lap={lap}')
        driver_telemetry = lap[1].get_telemetry().add_distance()
      
        driver_telemetry['Driver'] = driver
      
        driver_telemetry['Lap'] = lap[1]['LapNumber']
      
        driver_telemetry['Compound'] = lap[1]['Compound']
    
        telemetry = telemetry.append(driver_telemetry)


telemetry = telemetry[['Driver', 'Lap', 'Distance', 'Compound', 'Speed']]
telemetry['Compound'].loc[telemetry['Compound'] != 'INTERMEDIATE'] = 'SLICK'

minisectors_len = max(telemetry['Distance']) / 25
minisectors = []

for i in range(0, (24)):
    minisectors.append(minisectors_len * (i + 1))

average_speed = telemetry.groupby(['Lap', 'Compound'])['Speed'].mean().reset_index()

fastest_compounds = average_speed.loc[average_speed.groupby(['Lap'])['Speed'].idxmax()]
fastest_compounds = fastest_compounds[['Lap', 'Compound']].rename(columns={'Compound': 'Fastest_compound'})

print(fastest_compounds.to_string())


C:\Users\ssasi001\Anaconda3\lib\site-packages\fastf1\core.py:1220: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for Russian Grand Prix - Race [v2.3.0]
api            INFO 	Fetching driver list...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
api            INFO 	Fetching session status data...
api            INFO 	Fetching track status data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Fetching position data...
api            INFO 	Parsing position data...
api            INFO 	Fetching weather data...
api            INFO 	Fetching race control messages...
core           INFO 	Finished loading data for 20 drivers: ['44', '3

    Lap Fastest_compound
0    45            SLICK
2    46            SLICK
4    47            SLICK
6    48            SLICK
7    49     INTERMEDIATE
9    50     INTERMEDIATE
11   51     INTERMEDIATE
12   52     INTERMEDIATE


C:\Users\ssasi001\AppData\Local\Temp\ipykernel_6100\50044462.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)


In [79]:
from matplotlib import pyplot as plt
import random

race = ff1.get_session(2021, 'Russia', 'R')
quali = ff1.get_session(2021, 'Russia', 'Q')
practice3 = ff1.get_session(2021, 'Russia', 'FP3')
laps = race.load_laps(with_telemetry=True)
fastLaps = quali.load_laps(with_telemetry=True)

laps_nor = laps.pick_driver('NOR')
laps_ham = laps.pick_driver('HAM')
quali_nor = fastLaps.pick_driver('NOR')
quali_ham = fastLaps.pick_driver('HAM')

fastest_nor = laps_nor.pick_fastest()
fastest_ham = laps_ham.pick_fastest()
qualifying_nor = quali_nor.pick_fastest()
qualifying_ham = quali_ham.pick_fastest()

raceTime_nor = fastest_nor['LapTime']
raceTime_ham = fastest_ham['LapTime']
qualiTime_nor = qualifying_nor['LapTime']
qualiTime_ham = qualifying_ham['LapTime']
nor_avg = (raceTime_nor + qualiTime_nor)/2
ham_avg = (raceTime_ham + qualiTime_ham)/2
nor_comp = fastest_nor['Compound']
ham_comp = fastest_ham['Compound']
nor_tireLife = fastest_nor['TyreLife']
ham_tireLife = fastest_ham['TyreLife']
freshTire_nor = fastest_nor['TyreLife']
freshTire_ham = fastest_ham['TyreLife']

print(nor_tireLife, ham_tireLife)
print(nor_comp, ham_comp)
print(nor_avg, ham_avg)

lapTime_ham = []
lapTime_nor = []
laps = []
# 20 Lap Simulation
nor_time = 1.39
ham_time = 1.40
count_nor = 0
count_ham = 0
nor_tireLife = 11
ham_tireLife = 17
for i in range(0, 20):
    laps.append(i)
    if freshTire_nor and count_nor != 2:
        nor_time += 0.07
        lapTime_nor.append(nor_time)
        count_nor += 1
    if freshTire_ham and count_ham != 2:
        ham_time += 0.07
        lapTime_ham.append(ham_time)
        count_ham += 1
    
    if nor_tireLife < 15:
        nor_time -= 0.05
        lapTime_nor.append(nor_time)
    elif nor_tireLife > 25:
        nor_time += 0.15
        lapTime_nor.append(nor_time)
    elif nor_tireLife > 30:
        if random.random() > 0.5:
            nor_time += 0.10
            lapTime_nor.append(nor_time)
        else: 
            nor_time -= 0.10
            lapTime_nor.append(nor_time)
        
    if ham_tireLife < 15:
        ham_time -= 0.05
        lapTime_ham.append(ham_time)
    elif ham_tireLife > 25:
        ham_time += 0.15
        lapTime_ham.append(ham_time)
    elif nor_tireLife > 30:
        if random.random() > 0.5:
            ham_time += 0.10
            lapTime_ham.append(ham_time)
        else: 
            ham_time += 0.10
            lapTime_ham.append(ham_time)
    
    nor_tireLife += 1
    ham_tireLife += 1
    
print(lapTime_nor)
print(lapTime_ham)
print(nor_tireLife, ham_tireLife)

C:\Users\ssasi001\Anaconda3\lib\site-packages\fastf1\core.py:1220: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for Russian Grand Prix - Race [v2.3.0]
api            INFO 	Fetching driver list...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Fetching timing app data...
core           INFO 	Processing timing data...
api            INFO 	Fetching session status data...
api            INFO 	Fetching track status data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Fetching position data...
api            INFO 	Parsing position data...
api            INFO 	Fetching weather data...
api            INFO 	Fetching race control messages...
core           INFO 	Finished loading data for 20 drivers: ['44', '3

11.0 17.0
HARD HARD
0 days 00:01:39.708000 0 days 00:01:40.812500
[1.46, 1.41, 1.48, 1.43, 1.38, 1.3299999999999998, 1.4799999999999998, 1.6299999999999997, 1.7799999999999996, 1.9299999999999995, 2.0799999999999996]
[1.47, 1.54, 1.69, 1.8399999999999999, 1.9899999999999998, 2.1399999999999997, 2.2899999999999996, 2.4399999999999995, 2.5899999999999994, 2.7399999999999993, 2.8899999999999992, 3.039999999999999, 3.189999999999999]
31 37
